In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.shape

(10000, 14)

In [4]:
df.dtypes

RowNumber            int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
X = df.drop(['Surname', 'Exited', 'CustomerId', 'RowNumber'], axis=1)
X.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [6]:
X = pd.get_dummies(data=X, columns= ['Gender'], drop_first= True)


In [7]:
X = pd.get_dummies(data=X, columns= ['Geography'])
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,0,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,0,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,1,1,0,0
9996,516,35,10,57369.61,1,1,1,101699.77,1,1,0,0
9997,709,36,7,0.00,1,0,1,42085.58,0,1,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1,0


In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = scaler.fit_transform(X[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']])
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,0,1,0,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,0,1,0,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.842,0.283784,0.5,0.000000,0.333333,1,0,0.481341,1,1,0,0
9996,0.332,0.229730,1.0,0.228657,0.000000,1,1,0.508490,1,1,0,0
9997,0.718,0.243243,0.7,0.000000,0.000000,0,1,0.210390,0,1,0,0
9998,0.844,0.324324,0.3,0.299226,0.333333,1,0,0.464429,1,0,1,0


In [65]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [68]:
def ann(x, y):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33,stratify=y)
    print(type(X_train))
    model = keras.Sequential([
        keras.layers.Dense(50, input_shape = (12,), activation='relu'),
        #keras.layers.Dense(10, activation = 'relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
    model.fit(X_train, y_train, epochs = 10)
    model.evaluate(X_test, y_test)
    y_pred = model.predict(X_test)
    y_pred = np.rint(y_pred)
    print(y_pred[:20])
    print(y_test[:20])
    #print(confusion_matrix(y_train, y_pred))

In [54]:
y = df['Exited']
y.value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

In [69]:
X = np.array(X)
y = np.array(y)
ann(X, y)

<class 'numpy.ndarray'>
Epoch 1/10
210/210 [==============================] - 1s 1ms/step - loss: 0.1727 - accuracy: 0.7593
Epoch 2/10
210/210 [==============================] - 0s 1ms/step - loss: 0.1487 - accuracy: 0.7997
Epoch 3/10
210/210 [==============================] - 0s 1ms/step - loss: 0.1438 - accuracy: 0.8033
Epoch 4/10
210/210 [==============================] - 0s 1ms/step - loss: 0.1394 - accuracy: 0.8094
Epoch 5/10
210/210 [==============================] - 0s 1ms/step - loss: 0.1370 - accuracy: 0.8166
Epoch 6/10
210/210 [==============================] - 0s 1ms/step - loss: 0.1351 - accuracy: 0.8194
Epoch 7/10
210/210 [==============================] - 0s 1ms/step - loss: 0.1331 - accuracy: 0.8194
Epoch 8/10
210/210 [==============================] - 0s 1ms/step - loss: 0.1312 - accuracy: 0.8219
Epoch 9/10
210/210 [==============================] - 0s 1ms/step - loss: 0.1290 - accuracy: 0.8240
Epoch 10/10
210/210 [==============================] - 0s 974us/step
[[0.]
 